You'll need a few python packages to get this to work properly. A few important ones are loaded below

In [14]:
#imports
#imports 
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import time
import collections
from bs4 import BeautifulSoup
import re
from lxml import html
import platform
import time
from selenium import webdriver

You don't really need any of this next block other than the lists of team names, but I use it for selecting info from leagues and such

In [15]:
### CONTROLS ###
#these are things that should, basically, be fixed

league_mapping_dict = {'NL':['Phillies','Nationals','Mets','Marlins','Braves',
                         'Reds','Pirates','Cubs','Brewers','Cardinals',
                         'Giants','Dodgers','Rockies','Diamondbacks','Padres'],
                      'AL':['Yankees','Red Sox','Blue Jays','Rays','Orioles',
                           'White Sox','Tigers','Indians','Twins','Royals',
                           'Astros','Angels','Mariners','Athletics','Rangers']}

xr27_dict = {'1B':.5,'2B':.72,'3B':1.04,'HR':1.44,'BB':.34,'HBP':.34,'SB':.18,'CS':-.32,'OUTS':-.09,'SO':-.008,
            'GIDP':-.37} #mendoza values, keys will have to match df I create

positions = ['c','ss','2b','3b','cf','rf','lf','1b','dh'] #for fangraphs

caudillteams = ['A Cauhill','Corsa dei Torello','Dire Straits','Cubs Die Hard','Defying Gravity','Dodgers Dynasty',
        'Palo Altoids','Giants in their Field','Hormel Spam Fighters','Mercedes Bends','The Cleveland Experience',
        'The Cornballers']

robinsonteams = ["Washington-Robinson '05",'5th Sinco','All my XRs live in Texas','They Might Not Be Giants',
        'Atlanta Brewers','Bethesda Bird Watchers','Redwood City Otters','Encino Oaks','Break Dancing Giraffes',
        'Coney Island Steeplechase','Griffey Juniors','New York Flight','Narrows Lone Son','Moundpounders',
        'Metal Bleachers of Arlington','Larry Bowa Constrictors','Johnson ML',"Jobu's Lumber and Gas",
        "Eddie Murray's Moustache",'Peninsula Sea Horses','Pittsfield Electrics','Spahn and Sain and Pray for Rain',
        'TopDogs','Way Too Hot']

dotelteams = ['1st in WAR, 1st in peace...','Bartolo Colon and Friends','Beer Goggins','Lake Elsinore Stormy Daniels',
             'Mueller Light','Orans Crush','Par for the Coors','perennial bottomfeeders']

threefingerteams = ['Purple Eagles','Simply the Best','Old Gungho','Jacob Bay Area Giants',
                   'Hard Times Again','FW Kekiongas','Moneyball Still','Church of Baseball','PEDmart',
                   'Circular Karate Chop','Gray Panthers','Ragnar']

kearneyteams = ['Clean Coal Miners','Applesauce','Acorn Fielders','Its Not About the Money','Artful Dodgers',
               'Bat Girl',"Ware's The Fire",'Sweet Chin Music','Money Ball Too','A Kearney','Wide Awakes',"Say It Ain't Sosa"]



These inputs are important. They'd have to be replaced with your team, username, password, and the list of the team names in the league you wish to scrape

In [17]:
### INPUTS ###
myteam = "Washington-Robinson '05" #need to know my team for searching
myusername = 'luke.lavin' #replace with your username
mypassword = '' #replace with your password, I deleted mine from this cell
#because why would I give away my password!
teams = robinsonteams #choose league to look through

Connect to the Mendoza website. You will need to install phantomjs and the selenium webdriver to connect to the website with this code. I can include some instructions on that later if you like. You'll see the phantomjs path is local to my computer.

In [18]:
#set up ability to browse mendoza website by logging in
if platform.system() == 'Windows': #this assumes you're on a Windows Machine
    #PHANTOMJS_PATH = 'C:/Users/llavi/Anaconda4/Lib/site-packages/selenium/webdriver/phantomjs.exe'
    PHANTOMJS_PATH = 'C:/Users/llavi/OneDrive/Documents/Downloads/phantomjs-2.1.1-windows/bin/phantomjs.exe'
#else:
#    PHANTOMJS_PATH = './phantomjs'
print(PHANTOMJS_PATH)
browser = webdriver.PhantomJS(executable_path=PHANTOMJS_PATH)
#webdriver.PhantomJS(executable_path='Complete path/to/phantomjs')
browser.set_window_size(1366, 768)
browser.get("http://mendozabaseball.com/index.asp") #go to Mendoza webpage
browser.find_element_by_name("username").send_keys(myusername) #send Mendoza your username
browser.find_element_by_name("userpwd").send_keys(mypassword) #send Mendoza your password
browser.find_element_by_xpath("//input[@value='sign in']").click() #click to login
time.sleep(5)
html = browser.page_source
#print(html)
if 'Welcome' in html:
    print("You're logged in!")
else:
    print("Logging in failed. Perhaps, it was attempted with invalid credentials")


C:/Users/llavi/OneDrive/Documents/Downloads/phantomjs-2.1.1-windows/bin/phantomjs.exe
You're logged in!


If you're logged in you can run the code to scrape team data.

In [19]:
def get_mendoza_team_roster(team_key,league_team_name):
    print('grabbing data for '+str(team_key)+ ' while logged in as '+str(league_team_name))
    browser.get('http://mendozabaseball.com/manageroster.asp')
    browser.find_element_by_name("_change_team_teams").send_keys(league_team_name)
    browser.find_element_by_name("mendozaTeam").send_keys(team_key)
    html = browser.page_source
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    data_table = soup.find_all('table')[2]
    table_rows = data_table.find_all('tr')
    l = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td]
        l.append(row)
    #print(l[13:14])
    pd_columns = ['No.','Name','Salary','EndYear','Service','Status','XR','XR/27','RAP','RAVG','TRS','TRS/162','Pos','C','1B',
                  '2B','3B','SS','LF','CF','RF','SP','RP','Waive','Activate','Option']
    df = pd.DataFrame(l[13:], columns=pd_columns)
    df['Team'] = [team_key]*len(df.index)
    time.sleep(1.)
    return df

#myteam = '1st in WAR, 1st in peace...'
league_dfs = []
for t in teams:
    league_dfs.append(get_mendoza_team_roster(t,myteam))
league_df = pd.concat(league_dfs, ignore_index=True)

#clean up loaded league df player names so that they'll match better when needed
#this isn't perfect when players have nonalphanumeric characters in their name (e.g. Travis d'Arnaud)
new_names_list = []
for name in league_df['Name']:
    try:
        new_name = re.findall("[a-zA-Z\s.']+",str(name))[1].strip()+' '+re.findall("[a-zA-Z\s.']+",str(name))[0].strip()
        new_names_list.append(new_name)
    except IndexError:
        new_names_list.append(name)
league_df['orderedName'] = new_names_list

league_df

grabbing data for Washington-Robinson '05 while logged in as Washington-Robinson '05
grabbing data for 5th Sinco while logged in as Washington-Robinson '05
grabbing data for All my XRs live in Texas while logged in as Washington-Robinson '05
grabbing data for They Might Not Be Giants while logged in as Washington-Robinson '05
grabbing data for Atlanta Brewers while logged in as Washington-Robinson '05
grabbing data for Bethesda Bird Watchers while logged in as Washington-Robinson '05
grabbing data for Redwood City Otters while logged in as Washington-Robinson '05
grabbing data for Encino Oaks while logged in as Washington-Robinson '05
grabbing data for Break Dancing Giraffes while logged in as Washington-Robinson '05
grabbing data for Coney Island Steeplechase while logged in as Washington-Robinson '05
grabbing data for Griffey Juniors while logged in as Washington-Robinson '05
grabbing data for New York Flight while logged in as Washington-Robinson '05
grabbing data for Narrows Lone S

,No.,Name,Salary,EndYear,Service,Status,XR,XR/27,RAP,RAVG,...,LF,CF,RF,SP,RP,Waive,Activate,Option,Team,orderedName
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,Washington-Robinson '05,None
1,1,"Flowers, Tyler",$8.50,2020,7.148,A,36,4.42,0,0.00,...,,,,,,Waive,,,Washington-Robinson '05,Tyler Flowers
2,2,"Smith, Will",$.25,2019,0.000,A,34,6.87,0,0.00,...,,,,,,Waive,,Option,Washington-Robinson '05,Will Smith
3,3,"Votto, Joey",$26.00,2019,11.027,A,78,5.21,0,0.00,...,,,,,,Waive,,,Washington-Robinson '05,Joey Votto
4,4,"Astudillo, Willians",$.50,2019,0.061,A,21,3.86,0,0.00,...,,,,,,Waive,,Option,Washington-Robinson '05,Willians Astudillo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,3,"Mauricio, Ronny",$.25,2019,0.000,DES,0,0.00,0,0.00,...,,,,,,Purchase Contract,Release,None,Way Too Hot,Ronny Mauricio
1742,4,"Luciano, Marco",$.25,2019,0.000,DES,0,0.00,0,0.00,...,,,,,,Purchase Contract,Release,None,Way Too Hot,Marco Luciano
1743,5,"Alcantara, Kevin",$.25,2019,0.000,DES,0,0.00,0,0.00,...,,,,,,Purchase Contract,Release,None,Way Too Hot,Kevin Alcantara
1744,6,"Reyes, Jomar",$.25,2019,0.000,DES,0,0.00,0,0.00,...,,,,,,Purchase Contract,Release,None,Way Too Hot,Jomar Reyes


If you make it this far, you can take that loaded data and export it to a csv

In [13]:
league_df.to_csv('mymendozaleague.csv')